In [ ]:
from util import *

# Gaussian Clusters Example

In [ ]:
# Problem Setup --- Need to rework this example to be more general. I've made it so that the clusters are 
# in consecutive order for plotting m (0-99, 100-199). For this I had to re write the helper functions when 
# doing the Two moons dataset.

Ns = [100,100]
N = sum(Ns)
fs = 5
means = [np.array([1.0,0.]), np.array([-1.0,0.])]
Covs = [0.3*np.eye(2), 0.3*np.eye(2)]

X, W = generate_data_graphs(Ns, means, Covs)
w,v = get_eig_Lnorm(W)

tau, alpha, gamma = 0.001, 1.0, 0.0001
gamma2 = gamma**2.
B, labeled, ground_truth = other_setup(Ns,fs)
unlabeled = [i for i in range(N) if i not in labeled]
m, C, y = calc_orig(v, w, B, Ns, labeled, tau, alpha, gamma2)


sup1 = labeled[labeled < Ns[0]]
sup2 = labeled[labeled >= Ns[0]]
plt.rcParams['figure.figsize'] = (6,4)
plt.scatter(X[:100,0], X[:100,1], marker='x', c='k', alpha=0.2)
plt.scatter(X[100:,0], X[100:,1], marker='o', c='k',alpha=0.15)
plt.scatter(X[sup1,0], X[sup1,1], marker='x', c='b')
plt.scatter(X[sup2,0], X[sup2,1], marker='o', c='r', alpha=1.0)
plt.axis('equal')
plt.title('Dataset with First-Iter Supervision')
plt.show()

## Active Learning -- EEM 

In [ ]:
M = {}
M[0] = m
ERRS = []

#print('Iter = 0')
error, stats_obj = calc_stats(m, labeled, Ns)
ERRS.append(error)
#plot_iter(stats_obj, X, k_next=-1)



# AL choices
for l in range(20):
    print('Iter = %d' % (l + 1))
    k, m, C, y, labeled, unlabeled = run_next(m, C, y, labeled, unlabeled, ground_truth, gamma2)
    error, stats_obj = calc_stats(m, labeled, Ns)
    ERRS.append(error)
    plot_iter(stats_obj, X, k_next=k)
    M[l+1] = m


# Analyzing solution $\mathbf{m}$

In [ ]:
lin = [i for i in range(N)]
print(m.shape)
print(len(lin))
plt.scatter(lin, list(m), marker='.')
plt.plot(lin, N*[0.], 'r--', linewidth=1.5)
plt.title('Iter = %d' % l)
plt.show()

In [ ]:
m_new = m.copy()
m_new[m_new > 1.] = 1.
m_new[m_new < -1.] = -1.


In [ ]:
def E(x):
    vTx = v.T.dot(x.T)
    wvTx = np.diag(w).dot(vTx)
    val = vTx.T.dot(wvTx)
    return  val + (1./gamma**2)*np.linalg.norm(B.dot(x.T).flatten() - y)**2.

In [ ]:
E_m = E(m)
E_m_new = E(m_new)

In [ ]:
print(E_m, E_m_new)

In [ ]:
for l in range(5):
    if l % 3 == 0:
        m = list(M[l])
        lin = [i for i in range(N)]
        plt.scatter(lin, m, marker='.')
        plt.plot(lin, N*[0.], 'r--', linewidth=1.5)
        plt.title('Iter = %d' % l)
        plt.show()

## Compare to Random Sampling

In [ ]:
# Random sampling  choices
B, labeled, ground_truth = other_setup(Ns,fs)
unlabeled = [i for i in range(N) if i not in labeled]

m, C, y = calc_orig(v, w, B, Ns, labeled, tau, alpha, gamma2)

M = {}
M[0] = m
ERRS_rand = []

#print('Iter = 0')
error, stats_obj = calc_stats(m, labeled, Ns)
ERRS_rand.append(error)
#plot_iter(stats_obj, X, k_next=-1)


for l in range(20):
    #print('Iter = %d' % (l+1))
    
    k_next = unlabeled[np.random.choice(len(unlabeled),1)[0]]
    y_k_next = 1.
    if k_next >= Ns[0]:
        y_k_next = -1.

    m, C = calc_next_C_and_m(m, C, y, labeled, k_next, y_k_next, gamma2)
    y[k_next] = y_k_next
    labeled = np.array(list(labeled)+ [k_next])
    unlabeled.remove(k_next)
    
    error, stats_obj = calc_stats(m, labeled, Ns)
    ERRS_rand.append(error)
    #plot_iter(stats_obj, X, k_next=k_next)
    M[l+1] = m

In [ ]:
plt.plot(range(21),ERRS, 'b--', label='AL')
plt.plot(range(21), ERRS_rand, 'g', label='random')
plt.title('Convergence Plot')
plt.legend()
plt.show()

# 2 Moons Example


Functions in this section are defined in the ``util.py`` helper file with names like ``*2``, ``*2AL``, etc.

In [ ]:
from sklearn.datasets import make_moons

In [ ]:
# Setup
N = 1000
data = make_moons(N, noise=0.15)

X, ground_truth = data
X1, X2 = X[ground_truth == 0,:], X[ground_truth == 1,:] 
ground_truth[np.where(ground_truth == 0)] = -1

#plt.rcParams['figure.figsize'] =(6,6)
plt.scatter(X1[:,0], X1[:,1], c='r')
plt.scatter(X2[:,0], X2[:,1], c='b')
plt.title('Two Moons dataset')
plt.show()



# Define the labeled and unlabed indices
sup_percent = 0.1
indices = np.array(list(range(N)))
labeled_orig = set()
fid_orig = {}
gt_flipped = {}
for i in [-1,1]:
    i_mask = indices[ground_truth ==i]
    gt_flipped[i] = i_mask
    np.random.shuffle(i_mask)
    n_i = len(i_mask)
    fid_orig[i] = list(i_mask[:int(sup_percent*n_i)])
    labeled_orig = labeled_orig.union(set(fid_orig[i]))
unlabeled_orig = sorted(list(set(indices) - labeled_orig))
labeled_orig = sorted(list(labeled_orig))


# Graph Generation and Eigen-Calculation
W = make_sim_graph(X)
w,v = get_eig_Lnorm(W)


# Model Setup
tau, alpha, gamma = 0.001, 1.0, 0.0001
gamma2 = gamma**2.
verbose = True     # set to True if want to see visual of which points chosen
num_iters = 20

In [ ]:
# Active Learning Choices -- EEM 
labeled = labeled_orig.copy()
unlabeled = unlabeled_orig.copy()
fid = copy.deepcopy(fid_orig)

ERRS,M = run_test_AL(X, v, w, labeled, unlabeled, fid, ground_truth, gt_flipped, tau, alpha, gamma2, iters=num_iters, verbose=verbose)
if verbose:
    print(ERRS)

In [ ]:
# Random choice
labeled = labeled_orig.copy()
unlabeled = unlabeled_orig.copy()
fid = copy.deepcopy(fid_orig)

ERRS_rand, M_rand = run_test_rand(X, v, w, labeled, unlabeled, fid, ground_truth, gt_flipped, tau, alpha, gamma2, iters=num_iters, verbose=verbose)
if verbose:
    print(ERRS_rand)

## Error Plotting - 2 Moons 

In [ ]:
colors = ['y', 'r', 'k', 'g']
ids, errs = zip(*ERRS)
ids_rand, errs_rand = zip(*ERRS_rand)
plt.figure()
plt.plot([i for i in range(num_iters+1)], errs, 'b--', label='AL')
plt.plot([i for i in range(num_iters+1)], errs_rand, 'g-o', label='rand')
for i in range(4):
    labeled = labeled_orig.copy()
    unlabeled = unlabeled_orig.copy()
    fid = copy.deepcopy(fid_orig)

    ERRS_rand, M_rand = run_test_rand(X, v, w, labeled, unlabeled, fid, ground_truth, gt_flipped, tau, alpha, gamma2, iters=num_iters, verbose=False)
    ids_rand, errs_rand = zip(*ERRS_rand)
    plt.plot([i for i in range(num_iters+1)], errs_rand,'%so-'%colors[i], label='rand')
plt.legend()
